In [44]:
import sys
sys.path.append("/Users/bubble/Desktop/Project/Infrasound Sensor/Layout/Code/acousticsensor")

import gdsfactory as gf
import Tools.geo_trans as gt


In [45]:
# structure for each side
# single grid
L = [250, 500, 750, 1000]
g = [10, 20, 30]
w = 2
t = 0.5

k = (len(L) * len(g))
mesh = [None] * k
for i in range(k):
    mesh[i] = gf.Component()
side = gf.Component()
count = 0
origin = (15000 / 13, 0)
for i in range(4):
    for j in range(3):
        row = column =  L[i] // (g[j]+w)
        height = L[i] / row - w
        sg = gf.components.rectangle(size=(g[j], height), layer=(1, 0))
        for m in range(column):
            for n in range(row):
                sf_ref = mesh[count] << sg
                sf_ref.move((origin[0]+m*(g[j]+w), origin[1]+n*(height+w)))
        # add frame
        fra_width = column * (g[j] + w) + w
        frame = gf.components.rectangle(size=(fra_width, L[i]), layer=(1, 0))
        frame_ref = mesh[count] << frame
        frame_ref.movex(origin[0]-w)
        # add crystal
        mid = origin[0] - w + fra_width / 2
        cry_size = g[j] if j != 0 else 22
        crystal = gf.components.rectangle(size=(cry_size, cry_size), layer=(1, 0))
        crystal_ref = mesh[count] << crystal
        crystal_ref.move((mid-cry_size/2, L[i]-cry_size))
        # mesh array in one side
        mesh_ref = side << mesh[count]
        mesh_ref.movex(count * 15000 / 13)
        count += 1
        
side.show()

In [46]:
cell = gf.Component()
# backside etching 15mm * 15mm
backside = gf.components.rectangle(size=(15000, 15000), layer=(2, 0))
cell << backside
# cell.show()

Unnamed_358: ports ['e1', 'e2', 'e3', 'e4'], vinsts=[] info=Info() kcl=KCLayout(name='DEFAULT', layout=<klayout.dbcore.Layout object at 0x1269ff750>, layer_enclosures=LayerEnclosureModel(root={'78687732': LayerEnclosure(layer_sections={}, main_layer=WG (1/0), yaml_tag='!Enclosure'), '5c226817': LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure')}), cross_sections={'78687732_10400': SymmetricalCrossSection(width=10400, enclosure=LayerEnclosure(layer_sections={}, main_layer=WG (1/0), yaml_tag='!Enclosure'), name='78687732_10400'), '78687732_10000': SymmetricalCrossSection(width=10000, enclosure=LayerEnclosure(layer_sections={}, main_layer=WG (1/0), yaml_tag='!Enclosure'), name='78687732_10000'), '78687732_250000': SymmetricalCrossSection(width=250000, enclosure=LayerEnclosure(layer_sections={}, main_layer=WG (1/0), yaml_tag='!Enclosure'), name='78687732_250000'), '78687732_242000': SymmetricalCrossSection(width=242000, enclosure=LayerEnclosure(layer_section

In [47]:
# structure for each die

double_side = gf.Component()
# structure for double sides
side_ref = double_side << side
side_ref.dmirror_y(15000/2).dmirror_x(15000/2)
double_side << side

# structure for 4 sides
fside = gf.Component()
dside_ref = fside << double_side
dside_ref.drotate(angle=90, center=(15000/2, 15000/2))
fside << double_side
# fside.show()

Unnamed_360: ports [], vinsts=[] info=Info() kcl=KCLayout(name='DEFAULT', layout=<klayout.dbcore.Layout object at 0x1269ff750>, layer_enclosures=LayerEnclosureModel(root={'78687732': LayerEnclosure(layer_sections={}, main_layer=WG (1/0), yaml_tag='!Enclosure'), '5c226817': LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure')}), cross_sections={'78687732_10400': SymmetricalCrossSection(width=10400, enclosure=LayerEnclosure(layer_sections={}, main_layer=WG (1/0), yaml_tag='!Enclosure'), name='78687732_10400'), '78687732_10000': SymmetricalCrossSection(width=10000, enclosure=LayerEnclosure(layer_sections={}, main_layer=WG (1/0), yaml_tag='!Enclosure'), name='78687732_10000'), '78687732_250000': SymmetricalCrossSection(width=250000, enclosure=LayerEnclosure(layer_sections={}, main_layer=WG (1/0), yaml_tag='!Enclosure'), name='78687732_250000'), '78687732_242000': SymmetricalCrossSection(width=242000, enclosure=LayerEnclosure(layer_sections={}, main_layer=WG (1

In [48]:
# boolean operation
fside_ref = cell << fside
outside = gf.boolean(A = backside, B = fside_ref, operation="and", layer=(1, 0))
cell << outside
cell.show()
